In [ ]:
from post_processing import *
from const import ROOT
from const import *

In [ ]:
dicts = pickle.load(open("linked_list.p","rb"))
PAD=5

In [ ]:
#os.system("rm -r smoothing")
#os.system("mkdir smooth_vert")
#os.system("mkdir smooth_horz")
#os.system("rm -r smooth3")
os.system("mkdir smooth3")
counter = 0
coord_dict = {}
thresh = 4100
for i in tqdm(dicts):
    if i > 150: break
    if i not in [2,3,4,8,16,17,21,31,32,44,54,72,82,87,91,111,112,120,121,122,123,124,131,134,136,137,140,141,145,146,147,149]:
        idx1=0
        idx2=101
        idx3=0
        idx4=101
        jigsaw = dicts[i]
        vertical = len(jigsaw)
        horizontal = len(jigsaw[0])

        img=Image.open('raw/'+str(i)+".png").convert('L')
        arr= np.array(img)
        img2=Image.open('gt_smooth/'+str(i)+".png").convert('L')
        arr2=np.array(img2)
        
        while idx4 < 101*vertical: 
            idx1=0
            idx2=101
            while idx2 < 101*horizontal:
                patch = arr[idx1:idx2,idx3:idx4]
                gt = arr2[idx1:idx2,idx3:idx4]/178
                if (patch/255).sum()>thresh and gt.sum() > 0.18*101*101:
                    coord_dict[counter]=[i,[idx1,idx2,idx3,idx4]]
                    im = Image.fromarray(patch)
                    im.save("smooth3/"+str(counter)+".png")
                    counter=counter+1
                idx1=min(idx1+PAD,101*horizontal)
                idx2=min(idx2+PAD,101*horizontal)

            idx4=idx4+101
            idx3=idx3+101
        """   
        while idx2 < 101*horizontal: 
            idx3=0
            idx4=101
            while idx4 < 101*vertical:
                patch = arr[idx1:idx2,idx3:idx4]
                gt = arr2[idx1:idx2,idx3:idx4]/178
                if (patch/255).sum()>thresh and gt.sum() > 0.3*101*101:
                    coord_dict[counter]=[i,[idx1,idx2,idx3,idx4]]
                    im = Image.fromarray(patch)
                    im.save("smooth3/"+str(counter)+".png")
                    counter=counter+1
                idx3=min(idx3+PAD,101*horizontal)
                idx4=min(idx4+PAD,101*horizontal)

            idx1=idx2+101
            idx2=idx2+101
        #break
        #arr.shape
        """


            

In [ ]:
print(coord_dict)
pickle.dump(coord_dict,open("coord_dict.p","wb"))

In [ ]:
#coord_dict = pickle.load(open("coord_dict.p","rb"))
#s2 = pickle.load(open("2_smooth_ResNet34_res_25600079500_model.p","rb"))
sigmoids = pickle.load(open("avg_smoothed.p","rb")) 
#sigmoids = sigmoids/2

In [ ]:
del s0, s1#, s2

In [ ]:
del sigmoids

In [ ]:
import cv2
images={}
v = 0

for i in tqdm(dicts):
    if i > 150: break
    if i not in [2,3,4,8,16,17,21,31,32,44,54,72,82,87,91,111,112,120,121,122,123,124,131,134,136,137,140,141,145,146,147,149]:
        
        jigsaw = dicts[i]
        raw=Image.open('raw/'+str(i)+".png").convert('L')
        arr= np.array(raw)

        image = np.zeros(arr.shape)
        avgmask = np.zeros(arr.shape)
        horizontal = arr.shape[1]
        vertical = arr.shape[0]
        
        while(coord_dict[v][0]==i):
            patch = cv2.resize(sigmoids[v,:,:],(101,101))
            #print(patch.mean())
            
            idx1, idx2, idx3, idx4 = coord_dict[v][1]
            """
            print(image[idx1:idx2,idx3:idx4].shape)
            print(v)
            print(coord_dict[v])
            print(patch.shape)
            print(arr.shape)
            print("------")
            """
            #if v not in blacklist:
            
            if image[idx1:idx2,idx3:idx4].shape == patch.shape:
                image[idx1:idx2,idx3:idx4] = image[idx1:idx2,idx3:idx4]+patch

                avgmask[idx1:idx2,idx3:idx4] = avgmask[idx1:idx2,idx3:idx4]+1
            else:
                #print((idx2-idx1+1,idx4-idx3+1))
                #print(idx1, idx2, idx3, idx4)
                image[idx1:idx2,idx3:idx4] = image[idx1:idx2,idx3:idx4]+patch[0:min(idx2,vertical)-idx1,0:min(idx4,horizontal)-idx3]

                avgmask[idx1:idx2,idx3:idx4] = avgmask[idx1:idx2,idx3:idx4]+1
                
            #print(image.shape)
            #print(idx1,idx2,idx3,idx4)
            #print(patch)
            #break
            v=v+1
        #print("before, ",image.mean())
        for i1 in range(image.shape[0]):
            for i2 in range(image.shape[1]):
                image[i1][i2]=image[i1][i2]/max(avgmask[i1][i2],1)
        #print("after, ",image.mean())
        images[i]=image*255
        #coord_dict[0]
    #break

In [ ]:
pickle.dump(images,open("smoothed_sigmiods_pad10.p","wb"))

In [ ]:
images.keys()

In [ ]:
dfne = pd.read_csv("256_arith_mean.csv")
ct=0
from scipy import ndimage
visited=[]
for i in tqdm(dicts):
    if i > 148: break
    if i not in [2,3,4,8,16,17,21,31,32,44,54,72,82,87,91,111,112,120,121,122,123,124,131,134,136,137,140,141,145,146,147,149]:
        
        jigsaw = dicts[i]
        raw=Image.open('raw/'+str(i)+".png").convert('L')
        arr= np.array(raw)

        image = images[i]

        horizontal = arr.shape[1]
        vertical = arr.shape[0]
        
        for row in jigsaw:
            for col in row:
                id1 = jigsaw.index(row)
                id2 = row.index(col)
                
                patch = image[id1*101: (id1+1)*101, id2*101: (id2+1)*101]
                msum = (patch/255).sum() 
                if msum > 0.1*101*101 and id1 >1 and msum < 0.9*101*101:
                    if id1 >=2:
                        above = image[(id1-1)*101: (id1)*101, id2*101: (id2+1)*101]
                    if not(above[100,:].sum()!=0 and patch[0,:].sum()==0):
                        mask = ((patch/255)>0.45).astype('uint8')
                        #print(mask)
                        #mask = ndimage.binary_fill_holes(mask, structure=np.ones((5,5))).astype('uint8')
                        dfne.loc[dfne['id'] == col,'rle_mask']=' '.join(map(str, rle_encoding(mask)))  
                        ct=ct+1
                        visited.append(col)
                        #print((id1,id2))
                        #print(i)
                #else: 
                #    dfne.loc[dfne['id'] == col,'rle_mask']=' '.join(map(str, ""))  
                    
print("replacing {} masks".format(ct))
for i in tqdm(range(len(dfne))):
    #print(row[1]["rle_mask"] is float)
    try:
        s=0
        if type(dfne.iloc[i]["rle_mask"]) is float or dfne.iloc[i]["rle_mask"] == '' :
            #print("s")
            s=s+1
        #print(s)
        if s>0: 
            ct=ct+1
            #df2.loc[df2['id'] == df.iloc[i]['id'],'rle_mask']=' '.join(map(str, ""))
        else:
            if dfne.iloc[i]['id'] not in visited:
                dfne.loc[dfne['id'] == dfne.iloc[i]['id'],'rle_mask']=' '.join(map(str, ""))
    except ValueError:
        a=1                    
                
        
        


In [ ]:
dfne.to_csv("256_arith_mean_smoothed_only.csv")

In [ ]:
dfne = pd.read_csv("256_arith_mean.csv")
ct=0
from scipy import ndimage

for i in tqdm(dicts):
    if i > 148: break
    if i not in [2,3,4,8,16,17,21,31,32,44,54,72,82,87,91,111,112,120,121,122,123,124,131,134,136,137,140,141,145,146,147,149]:
        
        jigsaw = dicts[i]
        raw=Image.open('raw/'+str(i)+".png").convert('L')
        arr= np.array(raw)

        image = images[i]

        horizontal = arr.shape[1]
        vertical = arr.shape[0]
        
        for row in jigsaw:
            for col in row:
                id1 = jigsaw.index(row)
                id2 = row.index(col)
                
                patch = image[id1*101: (id1+1)*101, id2*101: (id2+1)*101]
                msum = (patch/255).sum() 
                if msum > 0.1*101*101 and msum < 0.9*101*101:
                    if id1 >=2:
                        above = image[(id1-1)*101: (id1)*101, id2*101: (id2+1)*101]
                    if not(above[100,:].sum()!=0 and patch[0,:].sum()==0):
                        mask = ((patch/255)>0.45).astype('uint8')
                        #print(mask)
                        #mask = ndimage.binary_fill_holes(mask, structure=np.ones((5,5))).astype('uint8')
                        dfne.loc[dfne['id'] == col,'rle_mask']=' '.join(map(str, rle_encoding(mask)))  
                        ct=ct+1
                        #print((id1,id2))
                        #print(i)
                #else: 
                #    dfne.loc[dfne['id'] == col,'rle_mask']=' '.join(map(str, ""))  
                    
print("replacing {} masks".format(ct))
dfne.to_csv("256_arith_mean_smoothed.csv")
                
        

In [ ]:
Image.fromarray((images[12]).astype("uint8"))

In [ ]:
df = pd.read_csv("0875_baseline.csv")
ct=0
for index in tqdm(images.keys()):
    image = np.array(images[index])/255
    ilist = dicts[index]
    height = len(ilist)
    width = len(ilist[0])
    
    for i in range(height):
        for j in range(width):
            raw=Image.open('raw/'+str(index)+".png").convert('L')
            arr= np.array(raw)
            im = image[i*101: (i+1)*101, j*101: (j+1)*101]
            if arr.sum()>4000 and  ilist[i][j] in test_file_list:
                new_code = rle_encoding( (im>0.4).astype('uint8') )
                old_code = df.loc[df['id'] == ilist[i][j]]['rle_mask'].values
                if len(old_code) != 0 and type(old_code[0]) is not float:
                    df.loc[df['id'] == ilist[i][j],'rle_mask']=' '.join(map(str, new_code)) 
                    #print("old: ",old_code)
                    #print("new: ",new_code)
                    #print("---------------")
                    ct=ct+1
            #demo = Image.fromarray((im*255).astype('uint8'))
            #demo.show()
            #break
        #break
    #print(len(ilist))
    #break
print("{} replacements".format(ct))

In [ ]:
df.to_csv("0875_baseline_1fold_smoothed.csv",index=False)

In [ ]:
images.keys()

In [ ]:
bp = (images[23]>0.4*255).astype('uint8')
Image.fromarray((bp*255))

In [ ]:
id1 =3
id2 =2
Image.fromarray((bp[ id1*101 : (id1+1)*101, id2*101 : (id2+1)*101]*255))

In [ ]:
images[10]*255

In [ ]:
blacklist=[]
#sums=[]
for j in tqdm(range(32580)):
    raw=Image.open('smoothing1/'+str(j)+".png").convert('L')
    arr= np.array(raw)/255
    #sums.append(arr.sum())
    if arr.sum()<4000: blacklist.append(j)

In [ ]:
len(blacklist)

In [ ]:
h=np.array(sums)

In [ ]:
h.mean()

In [ ]:
np.array(Image.open("gt_smooth/287.png").convert("L"))

In [ ]:
df=pd.read_csv("256_arith_mean_0871_leak4++_stage2+++.csv")


def is_vertical(img):
    diffs=[]
    if img.sum()==0: return False
    for i in range(101):
        if img[i,:].sum()==0 or img[i,:].mean()==255: return False
        diffs.append(np.square(img[0,:]-img[i,:]).mean())
    discrepancy=np.mean(np.array(diffs))
    #print(discrepancy)
    return discrepancy==0

def rle_decode(rle_mask,size):
    '''
    rle_mask: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = rle_mask.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(size*size, dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(size,size)

ct=0
from scipy import ndimage

masks =[]
for i in tqdm(dicts):
    per_im=0
    if i > 148: break
    if i not in [2,3,4,8,16,17,21,31,32,44,54,72,82,87,91,111,112,120,121,122,123,124,131,134,136,137,140,141,145,146,147,149]:
        
        jigsaw = dicts[i]
        raw=Image.open('raw/'+str(i)+".png").convert('L')
        arr= np.array(raw)

        image = images[i]

        horizontal = arr.shape[1]
        vertical = arr.shape[0]
        
        for id1 in range(vertical):
            for id2 in range(horizontal):
                patch = image[id1*101: (id1+1)*101, id2*101: (id2+1)*101]
                
                msum = (patch/255).sum() 
                if msum > 0.2*101*101 and msum < 0.9*101*101:

                    mask = ((patch/255)>0.45).astype('uint8')

                    old_code = df.loc[df['id'] == jigsaw[id1][id2]]['rle_mask'].values

                    if not(len(old_code) == 0 or type(old_code[0]) is float): 
                        old_code = old_code[0]

                        old_img = np.transpose(rle_decode(old_code,101))
                        #print(old_img.sum()/(101*101))
                        if old_img.sum() < 0.95*101*101 and (id1 <= 2 or not(is_vertical(old_img))) and not(is_vertical(np.rot90(mask))):
                        #if True:
                            #mask = ndimage.binary_fill_holes(mask, structure=np.ones((5,5))).astype('uint8')
                            #print(df.loc[df['id'] == jigsaw[id1][id2],'rle_mask'])
                            df.loc[df['id'] == jigsaw[id1][id2],'rle_mask']=' '.join(map(str, rle_encoding(mask)))  
                            masks.append([mask,i,id1,id2])
                            #print(df.loc[df['id'] == jigsaw[id1][id2],'rle_mask'])
                            ct=ct+1
                            per_im=per_im+1
                            #print((id1,id2))
                            #print(i)
        print("============================")
        print("Image: {} | {} masks replaced".format(i,per_im))  
        print("============================")
       
print("replacing {} masks".format(ct))
#dfne.to_csv("256_arith_mean_smoothed.csv")

In [ ]:
df.to_csv("256_arith_mean_0871_leak4++_stage2+++_smoothed.csv",index=False)

In [ ]:
Image.fromarray(masks[5][0]*255)

In [ ]:
masks[2][0][0:50,:].sum()

In [ ]:
len(masks)